In [14]:
#http://sujitpal.blogspot.com/2013/08/sentence-genre-classification-using.html

import matplotlib.pyplot as plt
import re
from textblob import TextBlob
import pandas
import sklearn
import cPickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.learning_curve import learning_curve
import pickle

In [9]:
def generate_xy(texts, labels):
  ftext = texts
  pipeline = Pipeline([
    ("count", CountVectorizer(stop_words='english', min_df=0.0,
              binary=False)),
    ("tfidf", TfidfTransformer(norm="l2"))
  ])
  X = pipeline.fit_transform(ftext)
  return X, y

In [3]:
# https://radimrehurek.com/data_science_python/

def split_into_lemmas(message):
    message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma
    return [word.lemma for word in words]

# Create dataset that we interface using pandas
dataset = []
with open('../data/label-abstract.txt') as f:
    data = f.read().split('\n\n')
    for example in data:
        components = re.split("\<[a-z]+\>", example)
        if len(components) > 1:
            idnum = components[1].strip()
            category = components[2].strip()
            abstract = components[3].strip().replace('\n', ' ')
            dataset.append((idnum, category, abstract))
df = pandas.DataFrame(data = dataset, columns= ['ID', 'Category', 'Abstract'])

                      ID  Category  \
0            0704.1711v2   stat.AP   
1            0704.3474v1   stat.AP   
2            0704.3862v1   stat.AP   
3            0705.0569v1   stat.AP   
4            0705.2515v1   stat.AP   
5            0705.3257v2   stat.AP   
6            0706.0073v1   stat.AP   
7            0706.1401v1   stat.AP   
8            0707.0462v1   stat.AP   
9            0707.0600v1   stat.AP   
10           0707.3013v1   stat.AP   
11           0708.0945v1   stat.AP   
12           0708.0974v1   stat.AP   
13           0708.1401v1   stat.AP   
14           0708.3545v1   stat.AP   
15           0708.3601v2   stat.AP   
16           0708.4318v1   stat.AP   
17           0708.4337v1   stat.AP   
18           0708.4350v1   stat.AP   
19           0708.4358v1   stat.AP   
20           0709.0165v1   stat.AP   
21           0709.0366v1   stat.AP   
22           0709.0394v1   stat.AP   
23           0709.0406v1   stat.AP   
24           0709.0421v1   stat.AP   
25          

In [18]:
X = df.Abstract
y = df.Category
X, y = generate_xy(X, y)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.1, random_state=42)

In [27]:
def train_model(X, y, binmodel):
  model = LinearSVC()
  model.fit(X, y)
  # reports
  ypred = model.predict(X)
  print "Confusion Matrix (Train):"
  print confusion_matrix(y, ypred)
  print "Classification Report (Train)"
  print classification_report(y, ypred)
  return model

def test_model(X, y, trained_model):
  model = trained_model
  if y is not None:
    # reports
    ypred = model.predict(X)
    print "Confusion Matrix (Test)"
    print confusion_matrix(y, ypred)
    print "Classification Report (Test)"
    print classification_report(y, ypred)

In [30]:
model = train_model(Xtrain, ytrain)
test_model(Xtest, ytest, model)

Confusion Matrix (Train):
[[27  0  0 ...,  0  0  0]
 [ 0 28  0 ...,  0  0  0]
 [ 0  0 27 ...,  0  0  0]
 ..., 
 [ 0  0  0 ..., 24  0  0]
 [ 0  0  0 ...,  0 28  0]
 [ 0  0  0 ...,  0  0 28]]
Classification Report (Train)
                    precision    recall  f1-score   support

          astro-ph       1.00      1.00      1.00        27
   cond-mat.dis-nn       1.00      1.00      1.00        28
 cond-mat.mes-hall       1.00      1.00      1.00        27
 cond-mat.mtrl-sci       1.00      1.00      1.00        26
    cond-mat.other       1.00      1.00      1.00        29
     cond-mat.soft       1.00      1.00      1.00        29
cond-mat.stat-mech       1.00      1.00      1.00        28
   cond-mat.str-el       1.00      1.00      1.00        29
 cond-mat.supr-con       1.00      1.00      1.00        28
             cs.AI       1.00      1.00      1.00        26
             cs.AR       1.00      1.00      1.00        25
             cs.CC       1.00      1.00      1.00        28

IOError: [Errno 2] No such file or directory: 'data/model.bin'